**input_vars**
6 times: 
bit_rate, buffer_size, rebuffering_time, bandwidth_measurement, chunk_til_video_end

In [ ]:
# input_range_dict = [(0.5, 1.5), (5.95, 6.1), (1.14, 1.15), (0.2, 0.22), (0.15, 0.25), (0.14, 0.3)]
# (0.5, 1.5), (1, 6), (0.5, 1.15), (0.1, 0.8), (0.15, 0.25), (0.14, 0.6)

input_range_dict = {
    1: [(0.8, 1.1), (2, 4), (0.7, 0.8), (0.5, 0.6), (0.12, 0.16), (0.3, 0.45)],
    2: [(0.6, 1.3), (3, 5), (0.6, 1.05), (0.3, 0.7), (0.13, 0.2), (0.22, 0.5)],
    3: [(0.5, 1.5), (1, 6), (0.5, 1.15), (0.1, 0.8), (0.15, 0.25), (0.14, 0.6)],
}

## Input range

## Proporty Definition

In [1]:
model_input_size = 48
model_output_size = 6

copy_one_argmax_candidate = 0
copy_two_argmax_candidate = 5

epsilon = 0.001
input_range_dict = {
    1: [(0.8, 1.1), (2, 4), (0.7, 0.8), (0.5, 0.6), (0.12, 0.16), (0.3, 0.45)],
    2: [(0.6, 1.3), (3, 5), (0.6, 1.05), (0.3, 0.7), (0.13, 0.2), (0.22, 0.5)],
    3: [(0.5, 1.5), (1, 6), (0.5, 1.15), (0.1, 0.8), (0.15, 0.25), (0.14, 0.6)],
}

DECLARATION_LINE_BASE = "(declare-const {} Real)"
INPUT_CONSTRAINT_LINE_BASE = "(assert ({} {} {}))"
OUTPUT_CONSTRAINT_LINE_BASE = "(assert (<= {} {}))"

def gen_prop(copy_one_argmax_candidate, copy_two_argmax_candidate, difficulty=1):
    lines = []
    lines.append("; variable declaration")
    for i in range(2*model_input_size):
        lines.append(DECLARATION_LINE_BASE.format(f"X_{i}"))

    for i in range(2*model_output_size):
        lines.append(DECLARATION_LINE_BASE.format(f"Y_{i}"))

    lines.append(" ")
    lines.append("; input constraints")
    for i in range(model_input_size):
        lines.append(INPUT_CONSTRAINT_LINE_BASE.format(">=", f"X_{i}", input_range_dict[difficulty][int(i/8)][0]))
        lines.append(INPUT_CONSTRAINT_LINE_BASE.format("<=", f"X_{i}", input_range_dict[difficulty][int(i/8)][1]))

    for i in range(model_input_size):
        lines.append(INPUT_CONSTRAINT_LINE_BASE.format(">=", f"X_{model_input_size+i}", 0.0))
        lines.append(INPUT_CONSTRAINT_LINE_BASE.format("<=", f"X_{model_input_size+i}", epsilon))

    lines.append(" ")
    lines.append("; output constraints")
    for i in range(model_output_size):
        if i == copy_one_argmax_candidate:
            continue
        lines.append(OUTPUT_CONSTRAINT_LINE_BASE.format(f"Y_{i}", f"Y_{copy_one_argmax_candidate}"))

    for i in range(model_output_size):
        if i == copy_two_argmax_candidate:
            continue
        lines.append(OUTPUT_CONSTRAINT_LINE_BASE.format(f"Y_{model_output_size+i}", f"Y_{model_output_size+copy_two_argmax_candidate}"))
    
    f = open("/home/mzi/sys-rl-verif/applications/pensieve/vnnlib/difficulty{}_eps{}_{}_{}.vnnlib".format(
        difficulty, str(epsilon)[2:], copy_one_argmax_candidate, copy_two_argmax_candidate), "w")
    f.write('\n'.join(lines) )
    f.close()


In [3]:
for difficulty in [1, 2, 3]:
    for i in [3, 4, 5]: # logit index: {0, 1, ..., 5}
        gen_prop(copy_one_argmax_candidate=0, copy_two_argmax_candidate=i, difficulty=difficulty)

## Model Processing

#### Pensieve

In [7]:
from onnx2pytorch import ConvertModel
import onnx 
import torch 
import torch.nn as nn

def get_model(size="small"):
    assert size in ["small", "mid", "big"]
    
    path_to_onnx_model = f"/home/mzi/sys-rl-verif/VNNComp23_NN4Sys/onnx/pensieve_{size}_simple.onnx"
    onnx_model = onnx.load(path_to_onnx_model)
    pytorch_model = ConvertModel(onnx_model)

    return pytorch_model


#### Comparative

In [8]:
def get_params_argmax(input_size):
    
    # Take sum of the input vars
    c01 = torch.zeros([1, 1, input_size+1])
    c01[0][0][0] = 1

    c02 = torch.zeros([1, 1, input_size+1])
    c02[0][0][0] = 1
    c02[0][0][-1] = 1

    return c01, c02

def pensieve() -> nn.Sequential:
    base_model = get_model()

    class MyModel(nn.ModuleList):
        def __init__(self, device=torch.device("cpu")):
            super(MyModel, self).__init__()

            input_size = 48
            self.input_size = input_size
            c01, c02 = get_params_argmax(input_size)
            
            self.ft = torch.nn.Flatten()

            #################
            # Model
            ################# 
            self.base_model = base_model
            
            #################
            # Input summation
            #################
            self.input_conv1 = nn.Conv1d(in_channels=1, out_channels=1, kernel_size=input_size+1)
            self.input_conv1.weight = torch.nn.Parameter(c01, requires_grad=True)
            self.input_conv1.bias = torch.nn.Parameter(torch.zeros_like(self.input_conv1.bias, requires_grad=True))
            
            self.input_conv2 = nn.Conv1d(in_channels=1, out_channels=1, kernel_size=input_size+1)
            self.input_conv2.weight = torch.nn.Parameter(c02, requires_grad=True)
            self.input_conv2.bias = torch.nn.Parameter(torch.zeros_like(self.input_conv2.bias, requires_grad=True))            

        def forward(self, obs):
            # input processing
            input1 = self.input_conv1(obs)
            input2 = self.input_conv2(obs)
            
            # the model
            copy1_logits = self.base_model(input1)
            copy2_logits = self.base_model(input2)
            
            return self.ft(torch.concat((copy1_logits, copy2_logits), dim=1))

    model = MyModel()

    return model
